In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import os
import re

from tqdm import tqdm
from transformers import *
from langdetect import detect

2023-04-24 01:52:35.267022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/envs/mixmind/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [2]:
import sys
sys.path.append('/home/wnsrb8560/MixMind/mm_bert/korean')

from kor_emotion_analysis import *
from kor_preprocessing import isKorean_percent

loading file vocab.txt from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at bert_ckpt/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,


In [3]:
import sys
sys.path.append('/home/wnsrb8560/MixMind/mm_bert/english') #

from eng_emotion_analysis import *

loading configuration file config.json from cache at /home/wnsrb8560/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/wnsrb8560/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
l

In [4]:
sentiment_model_kor = TFBertClassifier_KOR(model_name='bert-base-multilingual-cased', dir_path='bert_ckpt')
sentiment_model_eng = TFBertClassifier_ENG(model_name='bert-base-cased')

2023-04-24 01:52:45.817025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 01:52:45.844166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 01:52:45.844454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 01:52:45.845247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [5]:
# 가중치(한글)
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
sentiment_model_kor.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

/opt/conda/envs/mixmind/lib/python3.8/site-packages/tensorflow_addons/optimizers/rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
# 가중치(영어), this learning rate is for bert model , taken from huggingface website
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=5e-05, epsilon=1e-08,decay=0.01,clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.CategoricalAccuracy('balanced_accuracy'),
sentiment_model_eng.compile(optimizer = optimizer,loss = loss, metrics = metric)

In [7]:
sentiment_model_kor.load_weights('/home/wnsrb8560/MixMind/mm_bert/korean/korean_binary_sentiment_weights/korean_binary_sentiment_weights') #
sentiment_model_eng.load_weights('/home/wnsrb8560/MixMind/mm_bert/english/new_multi_sentiment_weights/new_multi_sentiment_weights') #

In [8]:
# 한글 사전 분류 위한 vocab
vocab_path = '/home/wnsrb8560/MixMind/mm_bert/korean/vocab_9class_500.csv'
stopwords_path = '/home/wnsrb8560/MixMind/mm_bert/korean/stopwords.txt'

vocab = load_vocab(vocab_path, stopwords_path)

In [9]:
# DB에서 리뷰 export
# import pymysql

# db = pymysql.connect(host='34.64.62.157', port=3306, user='', passwd='', dataset='mixmind_webservice', charset='utf8')

# cur = db.cursor()
# query = "Select id, review from mixmind_musicreview"

# review_df_from_db = pd.read_sql(query, db)

review_df_from_db = pd.read_csv('/home/wnsrb8560/MixMind/mm_bert/rock_youtube.csv', index_col=None)

In [10]:
review_df_from_db

,youtube_ID,review
0,BBdC1rl5sKY,"['수상할 정도로 천문학에 진심인 가수', '진짜 우리나라 팝 장르에서 밴드 음악을..."
1,Dqlr8EDunCM,"['진짜 예술가스럽네 이거 한국어버전도 나왔으면 좋겠다 노래좋다', '처음 멜론에서..."
2,OMjDI2NqQ9M,"['락발라드 시대가 다시오길 바랍니다', '너무너무 멋있습니다 노래에 감동 고음 ㅁ..."
3,Jh4QFaPmdss,['Soyeon is the No 1 artist in KPOP In terms o...
4,cFgk2PMgPJ4,['진짜 16강 딱 진출 확정됐을때 경기장에 이 노래 나왔으면 온몸에 전율이 흘렀겠...
...,...,...
3996,x6EITzLE0mQ,"['힐링프로세스는 진짜 범접할수없는 음반이다', '마지막 감당할 수 없는 외로움의 ..."
3997,uxdaTaMxkCs,"['세상에 알고리즘 역주행이 시작됐다 제노의 역습..', '베비와칭 보고 해장하라는..."
3998,1xyH_4JKvPQ,"['14년이 지난 2022년에 들어도 진짜 명곡이다.', '이 노래들으면 어릴적 피..."
3999,UkMHMEVaHrE,['정말 오랜만에 찾아 들었는데 너무 좋다..가사도 멜로디도. 이 때의 메이트 그립...


In [11]:
# 리뷰 전처리
for idx, row in enumerate(review_df_from_db.itertuples()):
    temp = list(set(row.review.lstrip('[').rstrip(']').split(',')))
    remove_set = {"'YouTube에서 로그아웃한 상태입니다'", " 'YouTube에서 로그아웃한 상태입니다'", " '로그인하면 동영상에 좋아요를 표시하고 댓글을 달거나 구독할 수 있습니다 '", "'로그인하면 동영상에 좋아요를 표시하고 댓글을 달거나 구독할 수 있습니다 '"}
    clear_temp = [i for i in temp if i not in remove_set]

    review_df_from_db.loc[idx, 'review'] = clear_temp

review_df_from_db

,youtube_ID,review
0,BBdC1rl5sKY,"[ '하 울 윤하 왜이렇게 이쁘지 음색대박 넘 잘부른다', '음색이 넘을수없는 사..."
1,Dqlr8EDunCM,"[ '이러니저러니해도 잘만든 음악인건 맞네', '이제 이 노래가 내 머릿속에 박혔..."
2,OMjDI2NqQ9M,[ '얀꺼 다 커버가자 음질 좋게 듣고 싶다 추억도 떠오르고 노랴도 잘부르시고 너무...
3,Jh4QFaPmdss,[ 'Congratulations to idle for winning digital...
4,cFgk2PMgPJ4,[ '오늘만 40번 불었다 고음이 맑아 들으면 들을 수록가사가 진짜 월드컵 응원 노...
...,...,...
3996,x6EITzLE0mQ,"[ '이게 어떻게 생각해다...쓰...벌...', 'Woah.. This is b..."
3997,uxdaTaMxkCs,"[ '이노래 이컨셉으로 데뷔를 했어야함...', '애니 얀데레 코디네 완전 ㄷ ㄷ..."
3998,1xyH_4JKvPQ,"[ '와칭 제노님이 도입부였다니', '0 03 미러폰이네... 처음으로 쓴폰 추억..."
3999,UkMHMEVaHrE,[ '도대체 무슨 말인건지.. 지금은 헤어져서 힘들더라도.. 조금 시간이 흐른후 그...


In [12]:
# 빈 벡터 데이터 프레임 생성
#predict_emotion = pd.DataFrame(index=range(0,0), columns=['id', 'love', 'joy', 'passion', 'happiness', 'sadness', 'anger', 'loneliness', 'longing', 'fear', 'surprise'])
#predict_emotion.to_csv('dance_predict_emotion.csv', index=False)


In [12]:
# 진행중이던 데이터프레임 불러오기
predict_emotion = pd.read_csv('rock_predict_emotion.csv', index_col=None)
predict_emotion

,id,love,joy,passion,happiness,sadness,anger,loneliness,longing,fear,surprise
0,0.0,0.46980,0.17499,0.07606,0.06590,0.04571,0.03942,0.01852,0.01913,0.00647,0.08401
1,1.0,0.45367,0.21996,0.07831,0.07392,0.04146,0.03856,0.02030,0.02080,0.00845,0.04457
2,2.0,0.52199,0.16882,0.08750,0.07374,0.04579,0.03923,0.02175,0.02875,0.00983,0.00261
3,3.0,0.57641,0.00564,0.00226,0.00181,0.00239,0.00003,0.00006,0.00021,0.00000,0.41118
4,4.0,0.38610,0.26727,0.08659,0.09471,0.05756,0.04342,0.02603,0.02417,0.01175,0.00241
...,...,...,...,...,...,...,...,...,...,...,...
2185,2185.0,0.39571,0.01596,0.00244,0.00367,0.00267,0.00249,0.00137,0.00142,0.00053,0.57373
2186,2186.0,0.46650,0.19623,0.09004,0.06830,0.04507,0.04077,0.02139,0.02316,0.00742,0.04114
2187,2187.0,0.41462,0.01090,0.00708,0.00352,0.00264,0.00239,0.00114,0.00136,0.00026,0.55608
2188,2188.0,0.43758,0.15247,0.10890,0.07111,0.06056,0.05005,0.02028,0.02548,0.00735,0.06621


In [ ]:
# predict_emotion.to_csv('predict_emotion.csv', index=False)

In [19]:
# 리뷰 구분 / 예측

for idx, row in zip(tqdm(range(len(predict_emotion), len(review_df_from_db))), review_df_from_db[len(predict_emotion):].itertuples()):
    temp_dict = {'love' : 0, 'fun': 0, 'enthusiasm' : 0, 'happiness' : 0, 'sadness' : 0, 'anger' : 0, 'loneliness' : 0, 'longing': 0, 'fear' : 0, 'surprise': 0}
    if row.review:
        for review in row.review:
            if isKorean_percent(review, 0.1) == 0: # 한글 10% 이상 있어야 한글로 취급, 아니면 알파벳으로 인식 
                try:
                    if detect(review) == 'en': # 알파벳 인식해도 스페인어나 이런게 섞인 경우가 있음. 영어로 인식하면 감정 처리.
                        eng_emotion = eng_emotion_predict(re.sub('[^a-zA-Z0-9]+', '', review), sentiment_model_eng)
                        for key1, val1 in eng_emotion.items():
                            temp_dict[key1] += val1
                except: # 스페인어 등은 패스
                    continue

            else:
                kor_emotion = hybrid_emotion_export_persent(re.sub('[^가-힣ㄱ-ㅎ0-9]+', '', review), sentiment_model_kor, vocab)
                # 영어, 한글 섞인 리뷰들 중 한글로 취급된 애들은 아예 한글만 보기 위해 알파벳 빼줌.
                for key2, val2 in kor_emotion.items():
                    temp_dict[key2] += val2
        
        if sum(temp_dict.values()) != 0:
            predict_emotion.loc[idx, 'id'] =  idx
            predict_emotion.loc[idx, 'love'] = np.round((temp_dict['love'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'joy'] = np.round((temp_dict['fun'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'passion'] = np.round((temp_dict['enthusiasm'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'happiness'] = np.round((temp_dict['happiness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'sadness'] = np.round((temp_dict['sadness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'anger'] = np.round((temp_dict['anger'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'loneliness'] = np.round((temp_dict['loneliness'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'longing'] = np.round((temp_dict['longing'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'fear'] = np.round((temp_dict['fear'] / sum(temp_dict.values())), 5)
            predict_emotion.loc[idx, 'surprise'] = np.round((temp_dict['surprise'] / sum(temp_dict.values())), 5)
            predict_emotion.to_csv('rock_predict_emotion.csv', index=False)
        
        else: # 리뷰에 스페인어 등만 있는, 영양가 없는 리뷰만 긁어서 temp_dict 합 값이 0이 될수 있음.
            predict_emotion[idx, 'id'] = idx
            predict_emotion.to_csv('rock_predict_emotion.csv', index=False)
            continue

    else: # 리뷰 빈 칸인 경우
        predict_emotion[idx, 'id'] = idx
        predict_emotion.to_csv('rock_predict_emotion.csv', index=False)
        continue

  0%|          | 0/2000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/envs/mixmind/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 39ms/step


  0%|          | 1/2000 [00:13<7:36:32, 13.70s/it]

1/1 [==============================] - 0s 40ms/step


  0%|          | 2/2000 [00:16<4:02:34,  7.28s/it]

...

1/1 [==============================] - 0s 38ms/step


  9%|▉         | 188/2000 [40:48<7:18:56, 14.53s/it]

1/1 [==============================] - 0s 37ms/step


  9%|▉         | 189/2000 [40:49<5:20:44, 10.63s/it]

1/1 [==============================] - 0s 40ms/step


In [ ]:
predict_emotion